In [1]:
import pandas as pd
from nsepy import get_history as gh
import datetime as dt
import yfinance as yf

In [2]:
import datetime as dt

start = dt.datetime(2021, 7, 1)
end = dt.datetime(2022, 1, 27)
symbol = 'AXISBANK.NS'  # Stock symbol in uppercase

stk_data = yf.download(symbol, start=start, end=end)


[*********************100%%**********************]  1 of 1 completed


In [3]:
stk_data=stk_data[["Open","High","Low","Close"]]
dataset=stk_data

In [4]:
#column="Close"

In [5]:
from sklearn.preprocessing import MinMaxScaler
Ms = MinMaxScaler()
data1= Ms.fit_transform(stk_data)
print("Len:",data1.shape)

Len: (143, 4)


In [6]:
data1=pd.DataFrame(data1,columns=["Open","High","Low","Close"])


In [7]:
training_size = round(len(data1 ) * 0.80)
print(training_size)
X_train=data1[:training_size]
X_test=data1[training_size:]
print("X_train length:",X_train.shape)
print("X_test length:",X_test.shape)
y_train=data1[:training_size]
y_test=data1[training_size:]
print("y_train length:",y_train.shape)
print("y_test length:",y_test.shape)

114
X_train length: (114, 4)
X_test length: (29, 4)
y_train length: (114, 4)
y_test length: (29, 4)


In [8]:
import warnings
warnings.filterwarnings("ignore")

In [9]:
performance={"Model":[],"RMSE":[],"MaPe":[],"Lag":[],"Test":[]}

In [10]:
def cominbation(dataset,listt):
    print(listt)
    datasetTwo=dataset[listt]
    test_obs = 28
    train =datasetTwo[:-test_obs]
    test = datasetTwo[-test_obs:]
    from statsmodels.tsa.api import VAR
    for i in [1,2,3,4,5,6,7,8,9,10]:
        model = VAR(train)
        results = model.fit(i)
        print('Order =', i)
        print('AIC: ', results.aic)
        print('BIC: ', results.bic)
        print()
    x = model.select_order(maxlags=12)
    order=x.selected_orders["aic"]
    result = model.fit(order)
    #result.summary()
    lagged_Values = train.values[-order:]
    pred = result.forecast(y=lagged_Values,steps=28) 
    preds=pd.DataFrame(pred,columns=listt)
    preds.to_csv("varforecasted_{}.csv".format(test_obs))
    from sklearn.metrics import mean_squared_error
    rmse= round(mean_squared_error(test,pred,squared=False))
    from sklearn.metrics import mean_absolute_percentage_error
    mape=mean_absolute_percentage_error(test,pred)
    performance["Model"].append(listt)
    performance["RMSE"].append(rmse)
    performance["MaPe"].append(mape)
    performance["Lag"].append(order)
    performance["Test"].append(test_obs)
    perf=pd.DataFrame(performance)
    return perf,result,pred

In [11]:
listt=["Close","High","Open","Low"]
#listt=["AQI_calculated","PM10","PM2.5","NOx","NO2","NO","NH3","SO2","CO",'year']


In [15]:
perf,result,pred=cominbation(data1,listt)

['Close', 'High', 'Open', 'Low']
Order = 1
AIC:  -28.077293879121825
BIC:  -27.59725906361402

Order = 2
AIC:  -28.20534686094131
BIC:  -27.336444547015255

Order = 3
AIC:  -28.039955787578794
BIC:  -26.77779559733464

Order = 4
AIC:  -27.852866761525966
BIC:  -26.192974385947238

Order = 5
AIC:  -27.683290249236762
BIC:  -25.621105242631643

Order = 6
AIC:  -27.57103431914593
BIC:  -25.101907821688002

Order = 7
AIC:  -27.47211616173052
BIC:  -24.591308547411913

Order = 8
AIC:  -27.593825360225566
BIC:  -24.296503807431442

Order = 9
AIC:  -27.45885022743061
BIC:  -23.740086209236402

Order = 10
AIC:  -27.32959530545205
BIC:  -23.184361996634586



In [16]:
data1

Open      High       Low     Close
0    0.508982  0.441549  0.543815  0.491495
1    0.497006  0.433854  0.540997  0.513145
2    0.544411  0.479275  0.593970  0.563144
3    0.543912  0.504095  0.604114  0.552835
4    0.538423  0.489203  0.587489  0.576289
..        ...       ...       ...       ...
138  0.385230  0.330107  0.398422  0.404639
139  0.414421  0.355920  0.417864  0.401031
140  0.369261  0.296848  0.338405  0.316495
141  0.334331  0.252916  0.226543  0.275773
142  0.329341  0.444527  0.341504  0.521134

[143 rows x 4 columns]

In [17]:
perf

Model  RMSE      MaPe  Lag  Test
0  [Close, High, Open, Low]    32  0.038810    2    28
1  [Close, High, Open, Low]     0  1.172438    2    28